
# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  

Created by C. Tranchant-Dubreuil (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD) - May 2022  SouthGreen training 

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022

Readapted using TransmittingScience teaching by C. Tranchant-Dubreuil (DIADE-IRD) and F. Sabot (DIADE-IRD) - October 2023
    
# <span style="color:#006E7F"> TP5. STRUCTURAL VARIANTS DETECTION USING LONG READS  </center> </span>

## __How to identify large variants using whole-genome assemblies ?__ 

</span>


### <span style="color:#006E7F">__0 - Preparing working environment__ <a class="anchor" id="work"></a></span>  
### <span style="color: #4CACBC;"> Create the working directory

In [115]:
mkdir -p /home/jovyan/work/RESULTS/SV
cd /home/jovyan/work/RESULTS/SV
ls -l

total 4
drwxr-xr-x 3 jovyan users 4096 Nov  2 17:35 SV_CALLING_SYRI


### <span style="color: #4CACBC;"> Use assemblies from AGGREGATED directory</span>  

In [116]:
ls ~/work/RESULTS/AGGREGATED/

4222_FLYE.fasta          B8_FLYE_MEDAKA.fasta   G11_FLYE_MEDAKA.fasta
4222_FLYE_MEDAKA.fasta   B8_RAVEN.fasta         G11_RAVEN.fasta
4222_RAVEN.fasta         B8_RAVEN_MEDAKA.fasta  G11_RAVEN_MEDAKA.fasta
4222_RAVEN_MEDAKA.fasta  BUSCO                  QUAST
B8_FLYE.fasta            G11_FLYE.fasta


#### Count the number of sequences of the fasta files in the directory `AGGREGATED` 

In [130]:
grep ">"  ~/work/RESULTS/AGGREGATED/*FLY*MEDAKA*fasta -c

/home/jovyan/work/RESULTS/AGGREGATED/4222_FLYE_MEDAKA.fasta:31
/home/jovyan/work/RESULTS/AGGREGATED/B8_FLYE_MEDAKA.fasta:24
/home/jovyan/work/RESULTS/AGGREGATED/G11_FLYE_MEDAKA.fasta:29


## <span style="color:#006E7F">I. SV calling using nucmer, syri <a class="anchor" id="SVsyri"></a></span>  

Siry can __ONLY__ be used if two reference genomes are same name and number of contigs. For this example we will use two assemblies with similar caracteristics

We will use :
* nucmer to align these two genomes. [nucmer manual](http://mummer.sourceforge.net/manual/)
* syri to detect SVs from nucmer alignment. [syri manual](https://schneebergerlab.github.io/syri/)
 

### <span style="color: #4CACBC;"> Initialization of two variables "genome"</span>  


#### Download assemblies

In [132]:
cd /home/jovyan/work/RESULTS/
wget https://itrop.ird.fr/sv-training/Assemblies.tar.gz
tar xvf Assemblies.tar.gz && rm Assemblies.tar.gz

--2023-11-02 18:09:24--  https://itrop.ird.fr/sv-training/Assemblies.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9150274 (8.7M) [application/x-gzip]
Saving to: ‘Assemblies.tar.gz’

Assemblies.tar.gz   100%[===================>]   8.73M  28.8MB/s    in 0.3s    

2023-11-02 18:09:25 (28.8 MB/s) - ‘Assemblies.tar.gz’ saved [9150274/9150274]

Assemblies/
Assemblies/assembly-stats.txt
Assemblies/A8_assembly.fasta
Assemblies/5417_assembly.fasta


In [133]:
more /home/jovyan/work/RESULTS/Assemblies/assembly-stats.txt

stats for 5417_assembly.fasta
sum = 15276926, n = 19, ave = 804048.74, largest = 1387673
N50 = 986735, n = 7
N60 = 911957, n = 9
N70 = 748028, n = 11
N80 = 734935, n = 13
N90 = 518210, n = 16
N100 = 233066, n = 19
N_count = 21334
Gaps = 4
-------------------------------------------------------------------------------
stats for A8_assembly.fasta
sum = 15306181, n = 19, ave = 805588.47, largest = 1359092
N50 = 945596, n = 8
N60 = 894004, n = 9
N70 = 781878, n = 11
N80 = 764371, n = 13
N90 = 523849, n = 15
N100 = 225373, n = 19
N_count = 0
Gaps = 0


#### Calculate the size of the genomes in the directory `Assemblies`  - seqtk comp 

Setk comp provides information on each sequence in the fasta file:
* column 1 contains the sequence name 
* column 2 corresponds to the sequence length in bp

In [135]:
seqtk comp ~/work/RESULTS/Assemblies/A8_assembly.fasta

Chr_19	225373	64696	47347	47449	65881	0	0	0	22630	0	0	0
Chr_18	308071	82805	71611	73071	80584	0	0	0	54144	0	0	0
Chr_17	474879	123835	113639	111999	125406	0	0	0	84218	0	0	0
Chr_16	512960	133382	122842	123591	133145	0	0	0	94466	0	0	0
Chr_15	523849	134394	126822	125045	137588	0	0	0	95424	0	0	0
Chr_14	781878	227795	164005	162339	227739	0	0	0	80494	0	0	0
Chr_13	770859	197275	190692	184663	198229	0	0	0	135176	0	0	0
Chr_12	720863	187269	174726	172799	186069	0	0	0	131280	0	0	0
Chr_11	764371	199672	183615	187446	193638	0	0	0	138776	0	0	0
Chr_10	791990	202169	192080	190618	207123	0	0	0	145556	0	0	0
Chr_9	894004	230073	217131	216824	229976	0	0	0	164058	0	0	0
Chr_8	945596	243332	229608	228567	244089	0	0	0	174160	0	0	0
Chr_7	1008950	260979	242999	244517	260455	0	0	0	181218	0	0	0
Chr_6	989488	255753	239132	239013	255590	0	0	0	180688	0	0	0
Chr_5	1016987	260225	248175	244438	264149	0	0	0	185334	0	0	0
Chr_4	1006439	261114	242606	244332	258387	0	0	0	183722	0	0	0
Chr_3	1079247	278475	261730	263373	275669

In [137]:
seqtk comp ~/work/RESULTS/Assemblies/5417_assembly.fasta

Chr_19	233066	66647	50144	48763	67512	0	0	0	24058	0	0	0
Chr_18	318881	85345	74297	75075	84164	0	0	0	55346	0	0	0
Chr_17	482895	126641	114374	113570	128310	0	0	0	83524	0	0	0
Chr_16	518210	136535	122232	123594	135749	0	0	100	91800	0	0	0
Chr_15	521692	134305	126076	124267	137044	0	0	0	94334	0	0	0
Chr_14	641580	186873	135344	135053	184310	0	0	0	71686	0	0	0
Chr_13	737724	188413	182726	176331	190154	0	0	100	129222	0	0	0
Chr_12	734935	191741	177466	175220	190508	0	0	0	132112	0	0	0
Chr_11	748028	195445	178757	183097	190587	0	0	142	134678	0	0	0
Chr_10	803017	206784	193589	192228	210416	0	0	0	144998	0	0	0
Chr_9	911957	235846	220425	219548	236138	0	0	0	164128	0	0	0
Chr_8	985652	255488	237567	237151	255446	0	0	0	178310	0	0	0
Chr_7	992004	250831	234160	234318	251703	0	0	20992	173692	0	0	0
Chr_6	986735	255731	237563	237449	255992	0	0	0	177994	0	0	0
Chr_5	1013628	260089	246867	243032	263640	0	0	0	183632	0	0	0
Chr_4	1016660	264661	243987	246425	261587	0	0	0	183968	0	0	0
Chr_3	1096451	283426	265347	2663

##### Compute the size of the genome 5417 using `seqtk comp | awk`

In [141]:
seqtk comp ~/work/RESULTS/Assemblies/A8_assembly.fasta | awk '{pb=pb+$2} END {print "Total pb :",pb}'

Total pb : 15306181


In [142]:
seqtk comp  ~/work/RESULTS/Assemblies/5417_assembly.fasta | awk '{pb=pb+$2} END {print "Total pb :",pb}'

Total pb : 15276926


In [143]:
reference_assembly="/home/jovyan/work/RESULTS/Assemblies/A8_assembly.fasta"
query_assembly="/home/jovyan/work/RESULTS/Assemblies/5417_assembly.fasta"

### <span style="color: #4CACBC;"> Create the working directory for SYRI analysis</span>  

In [144]:
mkdir -p ~/work/RESULTS/SV/SV_CALLING_SYRI/
cd ~/work/RESULTS/SV/SV_CALLING_SYRI/

In [145]:
pwd

/home/jovyan/work/RESULTS/SV/SV_CALLING_SYRI


### <span style="color: #4CACBC;"> Aligning genomes using `Nucmer` <a class="anchor" id="nucmer"></a></span>  

[Nucmer manual](http://mummer.sourceforge.net/manual/)

Some interesting parameters :
<code>
-maxmatch       Compute all maximal matches regardless of their uniqueness
-b|breaklen     Set the distance an alignment extension will attempt to extend poor scoring regions before giving up (default 200)
-c|mincluster   Sets the minimum length of a cluster of matches (default 65)
-l|minmatch     Set the minimum length of a single match (default 20)
</code>


In [146]:
nucmer --maxmatch $reference_assembly $query_assembly

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "out.ntref" of length 15306200
# construct suffix tree for sequence of length 15306200
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# process 153062 characters per dot
#...................................................................................................
# CONSTRUCTIONTIME /usr/bin/mummer out.ntref 4.88
# reading input file "/home/jovyan/work/RESULTS/Assemblies/5417_assembly.fasta" of length 15276944
# matching query-file "/home/jovyan/work/RESULTS/Assemblies/5417_assembly.fasta"
# against subject-file "out.ntref"
# COMPLETETIME /usr/bin/mummer out.ntref 18.31
# SPACE /usr/bin/mummer out.ntref 29.56
4: FINISHING DATA


In [147]:
ls -lrt

total 1788
-rw-r--r-- 1 jovyan users 1829444 Nov  2 18:17 out.delta


In [148]:
pwd

/home/jovyan/work/RESULTS/SV/SV_CALLING_SYRI


#### Check if new files have been generated by nucmer and display the first lines of the file `.delta`

In [149]:
head out.delta

/home/jovyan/work/RESULTS/Assemblies/A8_assembly.fasta /home/jovyan/work/RESULTS/Assemblies/5417_assembly.fasta
NUCMER
>Chr_18 Chr_19 308071 233066
1 110 2 111 0 0 0
0
1 110 23 132 0 0 0
0
1 110 233060 232951 0 0 0
0
1 110 233053 232944 0 0 0


#### Filtering nucmer results

We are going to remove small and lower quality alignments. Some interesting parameters :
<code>
-i float	Set the minimum alignment identity [0, 100], (default 0)
-l int		Set the minimum alignment length (default 0)
-m            Many-to-many alignment allowing for rearrangements (union of -r and -q alignments)
-q            Maps each position of each query to its best hit in the reference, allowing for reference overlaps
-r            Maps each position of each reference to its best hit in the query, allowing for query overlaps
</code>

Remove small and lower quality alignments


In [150]:
delta-filter -m -i 90 -l 100 out.delta > out.filtered.delta

#### Check that the new `delta`files has been filtered

In [151]:
ls -lrt

total 2440
-rw-r--r-- 1 jovyan users 1829444 Nov  2 18:17 out.delta
-rw-r--r-- 1 jovyan users  667053 Nov  2 18:19 out.filtered.delta


In [153]:
wc -l *.delta

 443088 out.delta
 218555 out.filtered.delta
 661643 total


#### Converting the file `delta` into a tabular file using `show-coords`

[show-coords manual](http://mummer.sourceforge.net/manual/#coords)
Some interesting parameters :
<code>
-c	Include percent coverage columns in the output
-d	Include the alignment direction/reading frame in the output (default for promer)
-H	Omit the output header
-I float	Set minimum percent identity to display
-l	Include sequence length columns in the output
-L int	Set minimum alignment length to display
-q	Sort output lines by query
-r	Sort output lines by reference
-T	Switch output to tab-delimited format
</code>



In [154]:
show-coords -Trd out.filtered.delta > out.filtered.withHeader.coords

In [155]:
show-coords -THrd out.filtered.delta > out.filtered.coords

In [156]:
ls -lrt

total 2640
-rw-r--r-- 1 jovyan users 1829444 Nov  2 18:17 out.delta
-rw-r--r-- 1 jovyan users  667053 Nov  2 18:19 out.filtered.delta
-rw-r--r-- 1 jovyan users  101108 Nov  2 18:20 out.filtered.withHeader.coords
-rw-r--r-- 1 jovyan users  100931 Nov  2 18:20 out.filtered.coords


In [157]:
head *.coords

==> out.filtered.coords <==
1	122	482889	482768	122	122	100.00	1	-1	Chr_1	Chr_17
394	689	228410	228116	296	295	92.23	1	-1	Chr_1	Chr_19
484	608	220395	220272	125	124	91.20	1	-1	Chr_1	Chr_19
495	689	226164	225970	195	195	92.82	1	-1	Chr_1	Chr_19
799	23710	1041	23963	22912	22923	99.41	1	1	Chr_1	Chr_1
6978	7128	1039	886	151	154	97.40	1	-1	Chr_1	Chr_1
23711	45447	24062	45783	21737	21722	98.05	1	1	Chr_1	Chr_1
26393	26528	451998	451864	136	135	97.06	1	-1	Chr_1	Chr_3
36290	36897	973523	974129	608	607	99.51	1	1	Chr_1	Chr_8
45590	49328	45931	49682	3739	3752	96.80	1	1	Chr_1	Chr_1

==> out.filtered.withHeader.coords <==
/home/jovyan/work/RESULTS/Assemblies/A8_assembly.fasta /home/jovyan/work/RESULTS/Assemblies/5417_assembly.fasta
NUCMER

[S1]	[E1]	[S2]	[E2]	[LEN 1]	[LEN 2]	[% IDY]	[FRM]	[TAGS]
1	122	482889	482768	122	122	100.00	1	-1	Chr_1	Chr_17
394	689	228410	228116	296	295	92.23	1	-1	Chr_1	Chr_19
484	608	220395	220272	125	124	91.20	1	-1	Chr_1	Chr_19
495	689	226164	225970	195	195	92.82	1	-1	Chr_1	

### <span style="color: #4CACBC;"> SV calling using `SYRI` <a class="anchor" id="siri"></a></span>  


In [160]:
syri -c out.filtered.coords -d out.filtered.delta -r $reference_assembly -q $query_assembly 

/opt/conda/envs/syri_env/lib/python3.9/multiprocessing/pool.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return list(map(*args))
/opt/conda/envs/syri_env/lib/python3.9/multiprocessing/pool.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return list(map(*args))
/opt/conda/envs/syri_env/lib/python3.9/multiprocessing/pool.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return list(map(*args))
/opt/conda/envs/syri_env/

In [162]:
ls -lt

total 59204
-rw-r--r-- 1 jovyan users     8996 Nov  2 18:22 syri.log
-rw-r--r-- 1 jovyan users      556 Nov  2 18:22 syri.summary
-rw-r--r-- 1 jovyan users 36898454 Nov  2 18:22 syri.vcf
-rw-r--r-- 1 jovyan users 21000671 Nov  2 18:22 syri.out
-rw-r--r-- 1 jovyan users   100931 Nov  2 18:20 out.filtered.coords
-rw-r--r-- 1 jovyan users   101108 Nov  2 18:20 out.filtered.withHeader.coords
-rw-r--r-- 1 jovyan users   667053 Nov  2 18:19 out.filtered.delta
-rw-r--r-- 1 jovyan users  1829444 Nov  2 18:17 out.delta


In [163]:
cat syri.summary

#Structural annotations
#Variation_type	Count	Length_ref	Length_qry
Syntenic regions	197	14079055	14107736
Inversions	32	397553	259052
Translocations	93	91312	90081
Duplications (reference)	152	79812	-
Duplications (query)	161	-	83493
Not aligned (reference)	372	667759	-
Not aligned (query)	413	-	738403


#Sequence annotations
#Variation_type	Count	Length_ref	Length_qry
SNPs	239492	239492	239492
Insertions	31092	-	114051
Deletions	38036	119912	-
Copygains	32	-	13466
Copylosses	28	17778	-
Highly diverged	707	488054	383996
Tandem repeats	23	8965	10360


In [164]:
head syri.vcf

##fileformat=VCFv4.3
##fileDate=20231102
##source=syri
##ALT=<ID=SYN,Description="Syntenic region">
##ALT=<ID=INV,Description="Inversion">
##ALT=<ID=TRANS,Description="Translocation">
##ALT=<ID=INVTR,Description="Inverted Translocation">
##ALT=<ID=DUP,Description="Duplication">
##ALT=<ID=INVDP,Description="Inverted Duplication">
##ALT=<ID=SYNAL,Description="Syntenic alignment">


In [169]:
tail syri.out

-	-	-	-	-	Chr_9	701140	701276	NOTAL776	-	NOTAL	-
-	-	-	-	-	Chr_9	735428	738835	NOTAL777	-	NOTAL	-
-	-	-	-	-	Chr_9	738993	739177	NOTAL778	-	NOTAL	-
-	-	-	-	-	Chr_9	739322	739749	NOTAL779	-	NOTAL	-
-	-	-	-	-	Chr_9	739861	757123	NOTAL780	-	NOTAL	-
-	-	-	-	-	Chr_9	757443	757592	NOTAL781	-	NOTAL	-
-	-	-	-	-	Chr_9	761215	761682	NOTAL782	-	NOTAL	-
-	-	-	-	-	Chr_9	821104	821384	NOTAL783	-	NOTAL	-
-	-	-	-	-	Chr_9	821595	822304	NOTAL784	-	NOTAL	-
-	-	-	-	-	Chr_9	911784	911854	NOTAL785	-	NOTAL	-


### <span style="color: #4CACBC;"> Extracting all SNP from syri  <a class="anchor" id="siri"></a></span>  

In [170]:
cat syri.out | grep SNP | cut -f 1,2 > a
cat syri.out | grep SNP | cut -f 4,5 > b
cat syri.out | grep SNP | cut -f 3 > c
awk '{ print $1+1 }' c > d
paste a d c > SNPs.bed
rm a b c d


In [171]:
echo -e $reference_assembly'\t ref' > plotsr_pos.txt
echo -e $query_assembly'\t query' >> plotsr_pos.txt

In [172]:
head plotsr_pos.txt

/home/jovyan/work/RESULTS/Assemblies/A8_assembly.fasta	 ref
/home/jovyan/work/RESULTS/Assemblies/5417_assembly.fasta	 query


In [173]:
plotsr --sr syri.out --genomes plotsr_pos.txt -s 500 -o plotsr.pdf -H 8 -W 5

2023-11-02 18:28:20,299 - Plotsr - INFO - Starting
2023-11-02 18:28:21,405 - Plotsr - INFO - Plot plotsr.pdf generated.
2023-11-02 18:28:21,405 - Plotsr - INFO - Finished



## <span style="color:#006E7F">II - Using assemblytics <a class="anchor" id="SVsyri"></a></span>  

* Download the file .delta generated by nucmer. 
* Load this file on the assemblytics website : [http://assemblytics.com/](http://assemblytics.com/)

---------------------

## <span style="color:#006E7F">II - III - using d-genies <a class="anchor" id="SVsyri"></a></span>  

* Download the two genomes (fasta files)
* Load these files on the d-genies website : [http://dgenies.toulouse.inra.fr/](http://dgenies.toulouse.inra.fr/)
